In [114]:
# Import data into pandas dataframe
import pandas as pd
df_train = pd.read_excel('relevance_train.xlsx')
df_test = pd.read_excel('relevance_test.xlsx')

       Unnamed: 0                                doc_id  judgement  \
0               0      004c6120d0aa69da29cc045da0562168          0   
1               1      005a1f0c2064487a7f15443b2a5f349a          0   
2               2  00722094-2935-11e2-b4e0-346287b7e56c          0   
3               3  007d2856-7cc4-11e4-84d4-7c896b90abdc          0   
4               4  009aafb6-0283-11e6-8bb1-f124a43f84dc          0   
...           ...                                   ...        ...   
21115       26228  fd63b1f8-f00e-11e4-a55f-38924fca94f9          0   
21116       26229  fdeefde0-44e1-11e4-b47c-f5889e061e5f          0   
21117       26230      fe320ce7929e640c70458009f73e5241          0   
21118       26231      ff1c523ea149f03d89019adb8782cdd9          0   
21119       26232  ff3a25b0-0ba4-11e4-8341-b8072b1e7348          0   

                  author                                               body  \
0         Terrence McCoy  <p>More than 60 years ago, a fair-skinned Iris...   
1

In [117]:
#MIGHT NEED TO UNCOMMENT SOME COMMANDS WHEN RUNNING CODE FOR FIRST TIME

# Import library for tokenisation
import nltk 
#Download punctuation infrormation
# nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
#Import regular expressions library
import re
#Download stopwords information
#nltk.download('stopwords')
from nltk.corpus import stopwords
#Import stemmer
from nltk.stem import PorterStemmer
#Import lemmatizer
from nltk.stem import WordNetLemmatizer
# nltk.download("wordnet")
# nltk.download("omw-1.4")

#Fetch stopwords in English
stopset = set(stopwords.words('english'))

# Define a function to remove stop words from a sentence 
def remove_stop_words(sentence): 
  # Split the sentence into individual words 
  words = sentence.split() 
  
  # List comprehension to remove stop words 
  filtered_words = [word for word in words if word not in stopset] 
  
  # Join the filtered words back into a sentence 
  return ' '.join(filtered_words)

#Define a function to stem all words in a sentence
ps = PorterStemmer()
def stem (sentence):
    stemmed = []
    # Split the sentence into individual words 
    words = sentence.split()
    for word in words:
        stemmed.append(ps.stem(word))
    #Join the stemmed words back into a sentence
    return ' '.join(stemmed)

#Define a function to find all lemmas of the words in a sentence
wl = WordNetLemmatizer()
def lemma (sentence):
    lemmas = []
    # Split the sentence into individual words 
    words = sentence.split()
    for word in words:
        lemmas.append(wl.lemmatize(word))
    #Join the stemmed words back into a sentence
    return ' '.join(lemmas)
        
#Define a function to perform language processing on dataframe collumn
def nlp (iterable):
    processed = []    
    for element in iterable:
        element = str(element)
        element = element.lower()
        element = ''.join([i for i in element if not i.isdigit()]) #Remove digits from string
        element = re.sub("\<.*?\>"," ", element) #Remove all HTML tags
        element = element.replace('\\n', '') #Remove new line token
        element = re.sub(r'[^\w\s]','',element) #Remove punctuation
        element = remove_stop_words(element) #Call remove stopwords function
        #element = lemma(element) #does not add much to the stemming
        element = stem(element)
        #Add processed element into the list
        processed.append(element)
    return processed


In [132]:
#For columns with classifiable text, create classification
#Note the numeric value is arbitrary so no numeric analysis is to be performed
def id_creator (iterable):
    id = []
    id_Dict = {}
    for item in iterable:
        item = str(item)
        if item not in id_Dict:
            id_Dict[item] = len(id_Dict) + 1
        id.append(id_Dict[item])
    return id


#Generate DataFrame with the processed text
processed_train = {'id': df_train['Unnamed: 0'], 
                   'author': df_train['author'], 
                   'title': nlp(df_train['title']),
                   'topic_id': df_train['topic_id'],
                   'body': nlp(df_train['body']),
                   'description_id': id_creator(df_train['description']),
                   'narrative_id': id_creator(df_train['narrative']),
                   'judgement':df_train['judgement']}
df_train_processed = pd.DataFrame(processed_train)

In [133]:
print(df_train_processed['description_id'])

0         1
1         1
2         1
3         1
4         1
         ..
21115    40
21116    40
21117    40
21118    40
21119    40
Name: description_id, Length: 21120, dtype: int64


In [137]:
#Generate DataFrame with the processed text for test 
descriptions = list(id_creator(df_train['description'].append(df_test['description'])))
description_test = descriptions[len(df_train['description']):]
narrative = list(id_creator(df_train['narrative'].append(df_test['narrative'])))
narrative_test = descriptions[len(df_train['narrative']):]

processed_test = {'id': df_test['Unnamed: 0'], 
                   'author': df_test['author'], 
                   'title': nlp(df_test['title']),
                   'topic_id': df_test['topic_id'],
                   'body': nlp(df_test['body']),
                   'description_id': description_test,
                   'narrative_id': narrative_test}
df_test_processed = pd.DataFrame(processed_test)

C:\Users\njb23192\AppData\Local\Temp\ipykernel_14652\2943614820.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  descriptions = list(id_creator(df_train['description'].append(df_test['description'])))
C:\Users\njb23192\AppData\Local\Temp\ipykernel_14652\2943614820.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  narrative = list(id_creator(df_train['narrative'].append(df_test['narrative'])))


In [139]:
#Export processed text to excel 
df_train_processed.to_excel('relevance_train_processed.xlsx')
df_test_processed.to_excel('relevance_test_processed.xlsx')